In [98]:
import os
import pandas as pd
from data_reader import *
from models import *
from tf_idf import *
from colorama import Fore
colorama.init(autoreset=True)

In [99]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))
file_path = os.path.join(parent_directory, "dataset/processed-full-name-dup.csv")
data = pd.read_csv(file_path)

parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))
file_path = os.path.join(parent_directory, "dataset/dsai-k66-gender-annotated-name.csv")
test_data = pd.read_csv(file_path)

In [100]:
print(data.head())
print("--------------------")
print(test_data.head())

           Full_Name  Gender
0      ngô xuân tùng       1
1  bùi dương thảo vy       0
2        lưu thế huy       1
3     nguyễn thị vân       0
4    dương minh long       1
--------------------
         Full_Name  Gender
0  Nguyễn Thanh An       1
1    Nguyễn Thế An       1
2        Lê Bá Anh       1
3       Vũ Lâm Anh       1
4     Ngô Việt Anh       1


In [101]:
RANDOM_STATE = 1989

In [ ]:
X = data["Full_Name"].values
y = data["Gender"].values

X_test = []
y_test = []

X_test = test_data["Full_Name"].values
y_test = test_data["Gender"].values

# Train test split
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(X_train)
print("--------------------")

# TF-IDF Tokenizer
TF_IDF = Compute_TF_IDF(X_train)
tfidf_X_train = TF_IDF.compute_tf_idf()
tfidf_X_test = TF_IDF.compute_tf_idf_for_test(X_test)
print(tfidf_X_train)
print("Shape:", tfidf_X_train.shape)
print("--------------------")

truncated_tfidf_X_train, truncated_tfidf_X_test = truncated_svd(tfidf_X_train, tfidf_X_test, random_state=RANDOM_STATE)
print(truncated_tfidf_X_train)
print("Shape:", truncated_tfidf_X_train.shape)

In [103]:
logreg = MyLogisticRegression(truncated_tfidf_X_train, truncated_tfidf_X_test, y_train, y_test, random_state=RANDOM_STATE,
                              regularization="lasso", regularization_strength=0.01)
logreg.fit()

LOGISTIC REGRESSION


In [ ]:
y_pred = logreg.predict()
convert_dict = {0: "Female", 1: "Male"}

print("{0:30}{1:10}{2:10}".format("Name", "True", "Predicted"))
print("{0:30}{1:10}{2:10}".format("----------", "--------", "--------"))
for i, name in enumerate(X_test):
    if y_test[i] == y_pred[i]: 
        print("{0:30}{1:10}{2:10}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]]))
    else:
        print("{0:30}{1:10}{2:10}{3:^11}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]], "ERROR"))

In [105]:
bnb = MyBernoulliNaiveBayes(tfidf_X_train, tfidf_X_test, y_train, y_test)
bnb.fit()

BERNOULLI NAIVE BAYES


In [ ]:
y_pred = bnb.predict()
convert_dict = {0: "Female", 1: "Male"}

print("{0:30}{1:10}{2:10}".format("Name", "True", "Predicted"))
print("{0:30}{1:10}{2:10}".format("----------", "--------", "--------"))
for i, name in enumerate(X_test):
    if y_test[i] == y_pred[i]: 
        print("{0:30}{1:10}{2:10}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]]))
    else:
        print("{0:30}{1:10}{2:10}{3:^11}".format(name, convert_dict[y_test[i]], convert_dict[y_pred[i]], "ERROR"))